<a href="https://colab.research.google.com/github/KGL8/Code-for-Lab/blob/main/photon_through_deuteron_target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import output
output.enable_custom_widget_manager()

In [3]:
!pip install pythreejs

import numpy as np
from pythreejs import *
import ipywidgets as widgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.7/271.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.3 MB/s eta 0:00:00


In [75]:
def ray(origin, direction, length, col):
    return Line(geometry=BufferGeometry(attributes={'position': BufferAttribute(array=np.array([origin, length * np.array(direction)]), normalized=False),}),material=LineBasicMaterial(color=col))

In [76]:
def point(position, col, size):
    return Points(geometry=BufferGeometry(attributes={"position": BufferAttribute(np.array([position]), normalized=False)}),
                  material=PointsMaterial(color=col, size=size, sizeAttenuation=False))

In [112]:
def box(width, height, depth, position, col):
    box_geometry = BoxBufferGeometry(width=width, height=height, depth=depth)

    box_material = MeshPhysicalMaterial(color=col, transparent=True, opacity=0.5)
    box_mesh = Mesh(geometry=box_geometry, material=box_material)

    wireframe_material = LineBasicMaterial(color='#ff7070',linewidth=2)
    wireframe = WireframeGeometry(box_geometry)
    wireframe_mesh = LineSegments(wireframe, wireframe_material)

    box_group = Group(children=[box_mesh, wireframe_mesh])
    box_group.position = position

    return box_group

In [79]:
# cartesian to spherical
def car_sph(x,y,z):
  r = np.sqrt(x**2 + y**2 + z**2)
  theta = np.arccos(z / r)
  phi = np.arctan2(y, x)
  return np.array([theta, phi, r])

In [80]:
# spherical to cartesian
def sph_car(theta, phi, r=1):
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return np.array([x, y, z])

In [81]:
def intersect_ray_box(ray_origin, ray_direction, box_min, box_max):
    pass

In [87]:
origin = 3
width = 2
height = 5
depth = 7
box_min = np.array([-width / 2, -height / 2, origin])
box_max = np.array([width / 2, height / 2, depth + origin])
dir = np.array([0.2,0,1])

In [115]:
target_box = box(width, height, depth, (0, 0, depth / 2 + origin), 'red')


ray_line = ray(np.array([0, 0, 0]), dir, 1000, '#f5f542')

z_ray = ray(np.array([0, 0, 0]), sph_car(0, 0), 1, 'red')
y_ray = ray(np.array([0, 0, 0]), sph_car(np.pi/2, np.pi/2), 1, 'green')
x_ray = ray(np.array([0, 0, 0]), sph_car(np.pi/2, 0), 1, 'blue')

origin_point = point([0, 0, 0], 'white', 4)

scene = Scene(children=[target_box, ray_line, origin_point, x_ray, y_ray, z_ray])
scene.background = '#141414'

lights = [
    DirectionalLight(color='white', intensity=3, position=[0, 200, 0]),
    DirectionalLight(color='white', intensity=3, position=[100, 200, 100]),
    DirectionalLight(color='white', intensity=3, position=[-100, -200, -100])
]
for light in lights:
    scene.add(light)

camera = PerspectiveCamera(position=[10, 10, 10], up=[0, 1, 0], children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

display(Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], antialias=True, width=800, height=600))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_ab28f964-db93-4b11-a9b8-2ade3c41b07d'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_17ff2a81-61fc-480f-8a9e-f4e2f9c7b69d'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_b7bbbb7f-b0a5-4383-9b8b-f87efd0be5bc'.

TraitError: The 'target' trait of a DirectionalLight instance expected an Uninitialized or an Object3D, not the str 'IPY_MODEL_19c601f4-2478-4883-8826-04e9157f1440'.